In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import shap
import matplotlib.pyplot as plt
import numpy as np



In [2]:
df12 = pd.read_parquet('ThermocoupleData_2024-12_v2.parquet')
df1 = pd.read_parquet('ThermocoupleData_2025-01_v2.parquet')
df2 = pd.read_parquet('ThermocoupleData_2025-02_v2.parquet')
df3 = pd.read_parquet('ThermocoupleData_2025-03_v2.parquet')
df4 = pd.read_parquet('ThermocoupleData_2025-04_v2.parquet')

dfall = pd.concat([df12, df1, df2, df3, df4], ignore_index=True)




In [3]:
# count of all the df rows
print(f"df12: {len(df12)}")
print(f"df1: {len(df1)}")
print(f"df2: {len(df2)}")
print(f"df3: {len(df3)}")
print(f"df4: {len(df4)}")
print(f"dfall: {len(dfall)}")


df12: 44641
df1: 44641
df2: 40321
df3: 44581
df4: 33121
dfall: 207305


In [4]:
dfall.columns

Index(['DateTime', 'AvgTemp_ZON1_M', 'AvgTemp_ZON2_M', 'AvgTemp_ZON3_M',
       'AvgTemp_ZON4_M', 'AvgTemp_ZON5_M', 'AvgTemp_ZON6_M',
       'LineControlHastSverk4_1Act', 'UgnZon1BransleFlodeAr_Over',
       'UgnZon1BransleFlodeAr_Under', 'UgnZon1OljaFlodeAr_FT131',
       'UgnZon1TempRegAr_TC1', 'UgnZon1TempSkyddAr_TC2',
       'UgnZon1TempVaggOverBandAr_TC3', 'UgnZon1TempVaggUnderBandAr_TC4',
       'UgnZon2BransleFlodeAr_Over', 'UgnZon2BransleFlodeAr_Under',
       'UgnZon2OljaFlodeAr_FT231', 'UgnZon2TempAr_TC1',
       'UgnZon2TempSkyddAr_TC2', 'UgnZon2TempVaggOverBandAr_TC3',
       'UgnZon2TempVaggUnderBandAr_TC4', 'UgnZon3BransleFlodeAr_Over',
       'UgnZon3BransleFlodeAr_Under', 'UgnZon3OljaFlodeAr_FT331',
       'UgnZon3TempRegAr_TC1', 'UgnZon3TempSkyddAr_TC2',
       'UgnZon3TempVaggAr_TC3', 'UgnZon3Temp_TC4_Ar', 'UgnZon3Temp_TC5_Ar',
       'UgnZon4BransleFlodeAr_Over', 'UgnZon4BransleFlodeAr_Under',
       'UgnZon4OljaFlodeAr_FT431', 'UgnZon4TempAr_TC1',
       'UgnZon4Tem

In [5]:
dfzon1 = dfall[['AvgTemp_ZON1_M', 'DateTime', 'UgnZon1BransleFlodeAr_Under', 'UgnZon1BransleFlodeAr_Over',
                 'UgnZon1OljaFlodeAr_FT131', 'UgnZon1TempRegAr_TC1', 'UgnZon1TempSkyddAr_TC2',
       'UgnZon1TempVaggOverBandAr_TC3', 'UgnZon1TempVaggUnderBandAr_TC4', 'LineControlHastSverk4_1Act']]

In [6]:
# Create lag features for dfzon1
# First, ensure the dataframe is sorted by DateTime
dfzon1_lag = dfzon1.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon1.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon1_lag[f"{col}_lag1"] = dfzon1_lag[col].shift(1)

# Reset index
dfzon1_lag = dfzon1_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon1 = dfzon1_lag.dropna().reset_index(drop=True)


In [7]:
dfzon2 = dfall[['AvgTemp_ZON2_M', 'DateTime', 'UgnZon2BransleFlodeAr_Under', 'UgnZon2BransleFlodeAr_Over',
                 'UgnZon2OljaFlodeAr_FT231', 'UgnZon2TempAr_TC1', 'UgnZon2TempSkyddAr_TC2',
         'UgnZon2TempVaggOverBandAr_TC3', 'UgnZon2TempVaggUnderBandAr_TC4', 'LineControlHastSverk4_1Act']]

In [8]:
# Create lag features for dfzon2
# First, ensure the dataframe is sorted by DateTime
dfzon2_lag = dfzon2.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon2.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon2_lag[f"{col}_lag1"] = dfzon2_lag[col].shift(1)

# Reset index
dfzon2_lag = dfzon2_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon2 = dfzon2_lag.dropna().reset_index(drop=True)


In [9]:
dfzon3 = dfall[['AvgTemp_ZON3_M', 'DateTime', 'UgnZon3BransleFlodeAr_Under', 'UgnZon3BransleFlodeAr_Over',
                    'UgnZon3OljaFlodeAr_FT331', 'UgnZon3TempRegAr_TC1', 'UgnZon3TempSkyddAr_TC2',
        'UgnZon3TempVaggAr_TC3', 'UgnZon3Temp_TC4_Ar', "UgnZon3Temp_TC5_Ar",  'LineControlHastSverk4_1Act']]

In [10]:
# Create lag features for dfzon3
# First, ensure the dataframe is sorted by DateTime
dfzon3_lag = dfzon3.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon3.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon3_lag[f"{col}_lag1"] = dfzon3_lag[col].shift(1)

# Reset index
dfzon3_lag = dfzon3_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon3 = dfzon3_lag.dropna().reset_index(drop=True)


In [11]:
dfzon4 = dfall[['AvgTemp_ZON4_M', 'DateTime', 'UgnZon4BransleFlodeAr_Under', 'UgnZon4BransleFlodeAr_Over',
                    'UgnZon4OljaFlodeAr_FT431', 'UgnZon4TempAr_TC1', 'UgnZon4TempSkyddAr_TC2',
        'UgnZon4TempVaggAr_TC3', 'UgnZon4TempVaggAr_TC4',  'LineControlHastSverk4_1Act']]

In [12]:
# Create lag features for dfzon4
# First, ensure the dataframe is sorted by DateTime
dfzon4_lag = dfzon4.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon4.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon4_lag[f"{col}_lag1"] = dfzon4_lag[col].shift(1)

# Reset index
dfzon4_lag = dfzon4_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon4 = dfzon4_lag.dropna().reset_index(drop=True)


In [13]:
dfzon5 = dfall[['AvgTemp_ZON5_M', 'DateTime',  
                    'UgnZon5OljaFlodeAr_FT531', 'UgnZon5TempAr_TC1', 'UgnZon5TempSkyddAr_TC2',
        'UgnZon5TempVaggAr_TC3', 'LineControlHastSverk4_1Act']]

In [14]:
# Create lag features for dfzon5
# First, ensure the dataframe is sorted by DateTime
dfzon5_lag = dfzon5.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon5.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon5_lag[f"{col}_lag1"] = dfzon5_lag[col].shift(1)

# Reset index
dfzon5_lag = dfzon5_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon5 = dfzon5_lag.dropna().reset_index(drop=True)


In [15]:
dfzon6 = dfall[['AvgTemp_ZON6_M', 'DateTime',
                    'UgnZon6OljaFlodeAr_FT631', 'UgnZon6TempAr_TC1', 'UgnZon6TempSkyddAr_TC2',
        'UgnZon6TempVaggAr_TC3', "UgnZon6TempUtgValvAr_TC5",  'LineControlHastSverk4_1Act']]

In [16]:
# Create lag features for dfzon6
# First, ensure the dataframe is sorted by DateTime
dfzon6_lag = dfzon6.sort_values('DateTime').copy()

# List of columns to create lag features for (all columns except DateTime)
columns_to_lag = [col for col in dfzon6.columns if col != 'DateTime']

# Create lag1 features for each column
for col in columns_to_lag:
    dfzon6_lag[f"{col}_lag1"] = dfzon6_lag[col].shift(1)

# Reset index
dfzon6_lag = dfzon6_lag.reset_index(drop=True)

# Note: The first row will have NaN values for all lag columns
# Drop the first row with NaN values if needed
dfzon6 = dfzon6_lag.dropna().reset_index(drop=True)


In [17]:
# concatenate dfzon1 and dfzon2 together for furnace1 df
df_furnace1 = pd.concat([dfzon1, dfzon2], axis=1)
# Furnace 2 = Zones 3, 4, 5, and 6
df_furnace2 = pd.concat([dfzon3, dfzon4, dfzon5, dfzon6], axis=1)

In [18]:
df_furnace1 = df_furnace1.loc[:,~df_furnace1.columns.duplicated()]
df_furnace2 = df_furnace2.loc[:,~df_furnace2.columns.duplicated()]

In [19]:
df_furnace1.columns

Index(['AvgTemp_ZON1_M', 'DateTime', 'UgnZon1BransleFlodeAr_Under',
       'UgnZon1BransleFlodeAr_Over', 'UgnZon1OljaFlodeAr_FT131',
       'UgnZon1TempRegAr_TC1', 'UgnZon1TempSkyddAr_TC2',
       'UgnZon1TempVaggOverBandAr_TC3', 'UgnZon1TempVaggUnderBandAr_TC4',
       'LineControlHastSverk4_1Act', 'AvgTemp_ZON1_M_lag1',
       'UgnZon1BransleFlodeAr_Under_lag1', 'UgnZon1BransleFlodeAr_Over_lag1',
       'UgnZon1OljaFlodeAr_FT131_lag1', 'UgnZon1TempRegAr_TC1_lag1',
       'UgnZon1TempSkyddAr_TC2_lag1', 'UgnZon1TempVaggOverBandAr_TC3_lag1',
       'UgnZon1TempVaggUnderBandAr_TC4_lag1',
       'LineControlHastSverk4_1Act_lag1', 'AvgTemp_ZON2_M',
       'UgnZon2BransleFlodeAr_Under', 'UgnZon2BransleFlodeAr_Over',
       'UgnZon2OljaFlodeAr_FT231', 'UgnZon2TempAr_TC1',
       'UgnZon2TempSkyddAr_TC2', 'UgnZon2TempVaggOverBandAr_TC3',
       'UgnZon2TempVaggUnderBandAr_TC4', 'AvgTemp_ZON2_M_lag1',
       'UgnZon2BransleFlodeAr_Under_lag1', 'UgnZon2BransleFlodeAr_Over_lag1',
       'UgnZon2

In [20]:
df_furnace2.columns

Index(['AvgTemp_ZON3_M', 'DateTime', 'UgnZon3BransleFlodeAr_Under',
       'UgnZon3BransleFlodeAr_Over', 'UgnZon3OljaFlodeAr_FT331',
       'UgnZon3TempRegAr_TC1', 'UgnZon3TempSkyddAr_TC2',
       'UgnZon3TempVaggAr_TC3', 'UgnZon3Temp_TC4_Ar', 'UgnZon3Temp_TC5_Ar',
       'LineControlHastSverk4_1Act', 'AvgTemp_ZON3_M_lag1',
       'UgnZon3BransleFlodeAr_Under_lag1', 'UgnZon3BransleFlodeAr_Over_lag1',
       'UgnZon3OljaFlodeAr_FT331_lag1', 'UgnZon3TempRegAr_TC1_lag1',
       'UgnZon3TempSkyddAr_TC2_lag1', 'UgnZon3TempVaggAr_TC3_lag1',
       'UgnZon3Temp_TC4_Ar_lag1', 'UgnZon3Temp_TC5_Ar_lag1',
       'LineControlHastSverk4_1Act_lag1', 'AvgTemp_ZON4_M',
       'UgnZon4BransleFlodeAr_Under', 'UgnZon4BransleFlodeAr_Over',
       'UgnZon4OljaFlodeAr_FT431', 'UgnZon4TempAr_TC1',
       'UgnZon4TempSkyddAr_TC2', 'UgnZon4TempVaggAr_TC3',
       'UgnZon4TempVaggAr_TC4', 'AvgTemp_ZON4_M_lag1',
       'UgnZon4BransleFlodeAr_Under_lag1', 'UgnZon4BransleFlodeAr_Over_lag1',
       'UgnZon4OljaFlod

In [21]:
df_furnace2.shape

(207295, 59)

In [ ]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# --- Configuration ---
AE_MODEL_DIR = "zone_models_final" # The folder with Autoencoder models
IF_MODEL_DIR = "zone_models_final_isoforest" # The folder with Isolation Forest models
PLOT_DIR = "benchmark_plots"
os.makedirs(PLOT_DIR, exist_ok=True)

all_data_map = {
    "furnace1": df_furnace1,
    "furnace2": df_furnace2,
    "zone1": dfzon1,
    "zone2": dfzon2,
    "zone3": dfzon3,
    "zone4": dfzon4,
    "zone5": dfzon5,
    "zone6": dfzon6,
}

# --- Main Evaluation Loop ---
print("--- Starting Benchmark Evaluation ---")

for name, df_full in all_data_map.items():
    print(f"\n--- Processing: {name} ---")
    
    # 1. Prepare Data: Drop non-numeric columns and handle NaNs
    df_model = df_full.drop(columns=['DateTime']).dropna()
    
    if len(df_model) < 2:
        print(f"⚠️ Insufficient data for {name}. Skipping.")
        continue
        
    # 2. Recreate the Test Set
    _, X_test_df = train_test_split(df_model, test_size=0.2, shuffle=False)

    if X_test_df.empty:
        print(f"⚠️ Test set is empty for {name}. Skipping.")
        continue

    # 3. Load All Saved Models and Scalers
    try:
        ae_model = load_model(os.path.join(AE_MODEL_DIR, f"{name}_autoencoder.keras"))
        ae_scaler = joblib.load(os.path.join(AE_MODEL_DIR, f"{name}_scaler.pkl"))
        if_model = joblib.load(os.path.join(IF_MODEL_DIR, f"{name}_isoforest.pkl"))
        if_scaler = joblib.load(os.path.join(IF_MODEL_DIR, f"{name}_scaler_if.pkl"))
    except FileNotFoundError as e:
        print(f"❌ Error loading model files for {name}: {e}. Skipping.")
        continue

    # 4. Generate Anomaly Scores for the Test Set
    # Autoencoder Scores
    X_test_scaled_ae = ae_scaler.transform(X_test_df)
    predictions_ae = ae_model.predict(X_test_scaled_ae, verbose=0)
    scores_ae = np.mean(np.square(X_test_scaled_ae - predictions_ae), axis=1)

    # Isolation Forest Scores
    X_test_scaled_if = if_scaler.transform(X_test_df)
    scores_if = -1 * if_model.score_samples(X_test_scaled_if)
    
    # 5. Normalize Scores to [0, 1] for Fair Comparison
    if (scores_ae.max() - scores_ae.min()) > 0:
        scores_ae_norm = (scores_ae - scores_ae.min()) / (scores_ae.max() - scores_ae.min())
    else:
        scores_ae_norm = np.zeros_like(scores_ae) 

    if (scores_if.max() - scores_if.min()) > 0:
        scores_if_norm = (scores_if - scores_if.min()) / (scores_if.max() - scores_if.min())
    else:
        scores_if_norm = np.zeros_like(scores_if)
        
    # 6. Calculate Correlation
    correlation = np.corrcoef(scores_ae_norm, scores_if_norm)[0, 1]
    
    # 7. Create and Save the Comparison Plot
    plt.figure(figsize=(15, 6))
    plt.plot(scores_ae_norm, label='Autoencoder Score (Normalized)', color='blue', alpha=0.9)
    plt.plot(scores_if_norm, label='Isolation Forest Score (Normalized)', color='red', alpha=0.7, linestyle='--')
    plt.title(f'Benchmark Comparison for: {name}\nCorrelation: {correlation:.2f}')
    plt.xlabel('Time (Test Set Samples)')
    plt.ylabel('Normalized Anomaly Score')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    plot_filename = os.path.join(PLOT_DIR, f"benchmark_{name}.png")
    plt.savefig(plot_filename)
    plt.close() 

    print(f"✅ Successfully processed {name}. Correlation: {correlation:.2f}. Plot saved to {plot_filename}")

print("\n--- Benchmark Evaluation Complete ---")

--- Starting Benchmark Evaluation ---

--- Processing: furnace1 ---
✅ Successfully processed furnace1. Correlation: 0.56. Plot saved to benchmark_plots\benchmark_furnace1.png

--- Processing: furnace2 ---
✅ Successfully processed furnace2. Correlation: 0.44. Plot saved to benchmark_plots\benchmark_furnace2.png

--- Processing: zone1 ---
✅ Successfully processed zone1. Correlation: 0.13. Plot saved to benchmark_plots\benchmark_zone1.png

--- Processing: zone2 ---
✅ Successfully processed zone2. Correlation: 0.11. Plot saved to benchmark_plots\benchmark_zone2.png

--- Processing: zone3 ---
✅ Successfully processed zone3. Correlation: 0.20. Plot saved to benchmark_plots\benchmark_zone3.png

--- Processing: zone4 ---
✅ Successfully processed zone4. Correlation: 0.10. Plot saved to benchmark_plots\benchmark_zone4.png

--- Processing: zone5 ---
✅ Successfully processed zone5. Correlation: 0.13. Plot saved to benchmark_plots\benchmark_zone5.png

--- Processing: zone6 ---
✅ Successfully proces